# Columns Selections
This is using NY data. Looks like inside aribnb data is same format so it can be used for other cities. This notebook is dealing with keeping/removing columns to use for the model.


In [1]:
import os
import io
import re
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read data, specify index so it's easier to join and search using loc
path = '../../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'listings.csv')

listings = pd.read_csv(listings_csv, index_col = 'id')

In [3]:
#shape
listings.shape

(50796, 105)

In [4]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

In [5]:
listings.head(1)

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2060,https://www.airbnb.com/rooms/2060,20200313233810,2020-03-14,Modern NYC,NaN,"Lovely, spacious, sunny 1 BR apartment in 6th Floor elevator building steps from gorgeous Fort Tryon Park/The Cloisters in Manhattan, less than 5 min walk from the A-train, 25-minutes to midtown Manhattan. Neighborhood is quiet, safe and hilly. Fully equipped, all brand new kitchen; king size bed; WiFi and cable; and more. We are looking for responsible renters who don't mind feeding our Maine Coon cat while staying there.","Lovely, spacious, sunny 1 BR apartment in 6th Floor elevator building steps from gorgeous Fort Tryon Park/The Cloisters in Manhattan, less than 5 min walk from the A-train, 25-minutes to midtown Manhattan. Neighborhood is quiet, safe and hilly. Fully equipped, all brand new kitchen; king size bed; WiFi and cable; and more. We are looking for responsible renters who don't mind feeding our Maine Coon cat while staying there.",none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/3755/3f9d31cf_original.jpg?aki_policy=large,NaN,2259,https://www.airbnb.com/users/show/2259,Jenny,2008-08-18,US,NaN,a few days or more,22%,50%,f,https://a0.muscache.com/im/users/2259/profile_pic/1259094383/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/2259/profile_pic/1259094383/original.jpg?aki_policy=profile_x_medium,Washington Heights,0.0,0.0,['reviews'],t,f,"New York, NY, United States",Manhattan,Washington Heights,Manhattan,New York,NY,10040,New York,"New York, NY",US,United States,40.85722,-73.9379,t,Other,Private room,2,NaN,1.0,2.0,Real Bed,"{Internet,Wifi}",NaN,$100.00,NaN,NaN,NaN,NaN,1,$0.00,1,730,1,1,730,730,1.0,730.0,4 weeks ago,t,30,60,90,365,2020-03-14,1,0,2008-09-22,2008-09-22,80.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,flexible,f,f,1,0,1,0,0.01


# Remove columns with less than 50% populated

In [6]:
#Create a funciton to check if columns have lots of null (50% or more)
def less_than_50_percent(colname):
    total_row = listings.shape[0]
    isnull_count = listings[colname].isna().sum()
    if isnull_count/total_row > .5:
        return True


In [7]:
columns = list(listings)
remove_columns_0 = []
for column in columns:
    remove_column_y_n = less_than_50_percent(column)
    if remove_column_y_n:
        remove_columns_0.append(column)

print(remove_columns_0)


['notes', 'thumbnail_url', 'medium_url', 'xl_picture_url', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names']


In [8]:
listings.drop(remove_columns_0, inplace=True, axis=1)
#chek if columns are removed
listings.shape


(50796, 96)

# Remove columns with only one value

In [9]:
#check if all records have same value
def all_same_value(colname):
    count_unique = len(listings[colname].unique())
    if count_unique == 1:
        return True
        

In [10]:
columns = list(listings)
remove_columns_1 = []
for column in columns:
    remove_column_y_n = all_same_value(column)
    if remove_column_y_n:
        remove_columns_1.append(column)

print(remove_columns_1)


['scrape_id', 'experiences_offered', 'country_code', 'country', 'has_availability', 'requires_license', 'is_business_travel_ready']


In [11]:
listings.drop(remove_columns_1, inplace=True, axis=1)
#chek if columns are removed
listings.shape


(50796, 89)

# Remove all objects columns
All text columns, check by lenght of max characters, excluding amenites column

In [12]:
#get max length
def getmaxlength(colname):
    listings['length'] = listings[colname].str.len()
    sorted = listings['length'].sort_values(ascending = False)
    maxlength = sorted.head(1).values[0]
    listings.drop('length', inplace=True, axis=1)
    return maxlength
    

In [13]:
#run this for all object type columns and exlcuding amenities column
#if it's more than 300, then remove them
columns = list(listings)
remove_columns_2 = []
for column in columns:
    if listings[column].dtypes == object and column != 'amenities':
        remove_column_y_n = getmaxlength(column)
        if remove_column_y_n > 300:
            remove_columns_2.append(column)

print(remove_columns_2)


['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'access', 'interaction', 'house_rules', 'host_about']


In [14]:
listings.drop(remove_columns_2, inplace=True, axis=1)
#chek if columns are removed
listings.shape


(50796, 80)

# Remove columns with only two values in where one value is more than 95%

In [15]:
#get ratio for colmuns with only two values
def countratio(colname):
    temp_list = listings.groupby(colname)[colname].count().tolist()
    if len(temp_list) == 2:
        row_count = listings.shape[0]
        lowest_count = min(temp_list)
        ratio = lowest_count/row_count
        return ratio
        

In [16]:
#run this for all columns
#if it's less than 5%, then remove them
columns = list(listings)
remove_columns_3 = []
for column in columns:
    remove_column_y_n = countratio(column)
    try:
        if remove_column_y_n < .05:
            remove_columns_3.append(column)
    except: ''

print(remove_columns_3)


['last_scraped', 'host_has_profile_pic', 'calendar_last_scraped', 'require_guest_profile_picture', 'require_guest_phone_verification']


In [17]:
listings.drop(remove_columns_3, inplace=True, axis=1)
#chek if columns are removed
listings.shape


(50796, 75)

# Remove columns with url to pictures and thumbnail

In [18]:
#create list of picutres and thumbnail columns
remove_columns_4 = []
for col in listings.filter(like='picture').columns:
    remove_columns_4.append(col)

for col in listings.filter(like='thumbnail').columns:
    remove_columns_4.append(col)

remove_columns_4


['picture_url', 'host_picture_url', 'host_thumbnail_url']

In [19]:
listings.drop(remove_columns_4, inplace=True, axis=1)
#chek if columns are removed
listings.shape


(50796, 72)

# Remove unused columns
May use it later but remove for now

In [20]:
Unused_columns = ['listing_url', 'host_location', 'name', 'host_id', 'host_url', 'host_name', 'host_total_listings_count', 'city', 'street', 'state', 'zipcode', 'neighbourhood', 'host_neighbourhood', 'neighbourhood_cleansed', 'market', 'smart_location', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
'availability_60', 'availability_90', 'availability_365', 'latitude', 'longitude']

listings.drop(columns=Unused_columns, inplace = True)


# Export data to another csv
Filename is selected_columns_listings.csv. To be used by next program

In [21]:
#export columns left to csv to be used with second notebook
path = '../../../data/new-york-city-airbnb-open-data/'

listings_csv = os.path.join(path,'selected_columns_listings.csv')
listings.to_csv(listings_csv)


In [22]:
listings.shape

(50796, 39)